# 5장. GPU 효율적인 학습

- **GPU와 딥러닝 연산**  
  - 딥러닝 모델은 많은 행렬 곱셈 연산을 수행하며, GPU는 병렬 처리가 가능하여 연산 속도를 높이는 데 활용됨.  
  - LLM(대형 언어 모델) 등장이후 모델 크기 증가로 인해 단일 GPU로는 학습이 어려워짐.  
  - GPU는 가격이 비싸 일반 기업 및 개인이 활용하기 어려움.  
  - 이를 해결하고 GPU를 효율적으로 활용하는 기술이 발전하고 있음.  

- **GPU 효율적 활용 기술**  
  - 저장 및 연산에 사용되는 데이터 타입 검토.  
  - GPU 메모리 효율적 사용 방법:
    - **Gradient Accumulation** (그래디언트 누적)  
    - **Gradient Checkpointing** (그래디언트 체크포인팅)  
  - 다중 GPU 활용한 **분산 학습** 방식.  
  - **DeepSpeed ZeRO**: 분산 학습 시 메모리 비효율 문제 해결.  

- **LoRA(Low Rank Adaptation) 및 QLoRA**  
  - 전체 모델이 아닌 **일부만 업데이트**하여 학습 성능 유지하면서 메모리 사용량 절감.  
  - **QLoRA(Quantized LoRA)**: 모델을 낮은 비트로 저장해 GPU 메모리 효율 향상.


- **사용할 라이브러리 설치**

In [1]:
# !pip install transformers==4.40.1 datasets==2.19.0 accelerate==0.30.0 peft==0.10.0 bitsandbytes==0.43.1 -qqq

- **라이브러리 임포트**

In [2]:
# 필요한 라이브러리들을 임포트
import transformers  # 트랜스포머 모델을 위한 라이브러리
import datasets     # 데이터셋 처리를 위한 라이브러리
import accelerate   # GPU 가속을 위한 라이브러리
import peft        # 파라미터 효율적 미세조정을 위한 라이브러리
import bitsandbytes # 양자화 및 최적화를 위한 라이브러리

# 경고 메시지 숨기기
import warnings
warnings.filterwarnings('ignore')

---

## GPU에 올라가는 데이터 살펴보기

- **OOM(Out of Memory) 오류**  
  - GPU 메모리가 가득 차서 추가적인 데이터를 저장하지 못할 때 발생하는 오류.  
  - 딥러닝 모델 학습 및 추론 시 흔히 발생하는 문제.  

- **GPU 메모리에 올라가는 데이터**  
  - **딥러닝 모델 자체** (모델의 가중치 및 파라미터).  
    - **파라미터의 데이터 형식** (정수 또는 부동소수점 숫자로 저장됨).  
    - **양자화(Quantization)**: 모델의 크기를 줄이기 위해 숫자 표현 방식을 변경하는 기법.  
  - 학습 및 추론을 위한 **입력 데이터 및 중간 연산 결과**.  
  - **그래디언트 및 옵티마이저** 관련 데이터 (학습 시 필요).  

- **메모리 사용량 분석 및 계산**  
  - GPU 메모리 사용량을 줄이기 위한 최적화 방법 검토.  
  - 허깅페이스에서 제공하는 **메모리 계산 기능**을 활용하여 학습 및 추론 시 필요한 메모리를 예측.

---

### 딥러닝 모델의 데이터 타입

- **부동소수점 데이터 형식**  
  - 기본적으로 **float32(32비트 부동소수점)** 사용.  
  - 정밀한 계산이 필요하면 **float64(64비트 부동소수점)** 사용.  
  - 비트 수가 증가할수록 표현할 수 있는 범위와 정밀도가 증가하지만 메모리 사용량도 증가.  

- **LLM(대형 언어 모델)과 데이터 형식**  
  - LLM의 크기는 사용되는 **파라미터 개수 × 데이터 비트 수**에 따라 결정됨.  
  - 과거에는 **float32(4바이트)** 사용했으나, 파라미터 증가로 인해 메모리 부담이 커짐.  
  - 최근에는 **fp16(16비트 부동소수점)** 또는 **bf16(brain float 16)** 사용이 일반적.  


>**부동소수점의 구조포**
>
>$\text{Value} = (-1)^{\text{Sign}} \times (\text{Mantissa}) \times 2^{\text{Exponent} - \text{Bias}}$
>
>- **부호 비트 (Sign bit)**: $ (-1)^{\text{Sign}} $
>   - 1비트로 표현되며, 0이면 양수(+), 1이면 음수(-)입니다.
>
>- **지수 (Exponent, 指數)**: $ 2^{\text{Exponent} - \text{Bias}} $
>   - 숫자의 크기를 결정하며, 특정 **Bias(편향값)** 를 더한 값으로 저장됩니다.
>   - Bias는 단정도(32비트)에서는 127, 배정도(64비트)에서는 1023입니다.
>
>- **가수 (Mantissa, 仮數)**: $ \text{Mantissa} $
>   - 실수의 정확도를 결정하며, **정규화된 2진수** 형태에서 소수점 이하 부분만 저장됩니다.
>   - 첫 번째 정수 부분인 **1**은 항상 존재하므로 저장하지 않으며, 이를 **암시적 비트 (Implicit Bit)** 라고 합니다.

  <img src="./images/fig_05_01.png" width="800">

- **fp32, fp16, bf16 비교**  
  - **fp32**:  
    - 지수부(Exponent) 8비트, 가수부(Mantissa) 23비트.  
    - 넓은 범위와 높은 정밀도를 제공.  
  - **fp16**:  
    - 지수부 5비트, 가수부 10비트.  
    - 표현할 수 있는 수의 범위와 정밀도가 **fp32보다 낮음**.  
    - 정밀도 문제로 인해 일부 계산에서 정확도가 떨어질 수 있음.  
  - **bf16**:  
    - 지수부 8비트, 가수부 7비트.  
    - **fp32와 같은 지수 범위**를 가지면서도 메모리 사용량 절감 가능.  
    - 딥러닝 연산에서 fp16보다 안정적인 결과를 제공.  

- **모델 크기 및 GPU 메모리 사용량 계산**  
  - 모델 크기 = **파라미터 개수 × 데이터 타입의 바이트 크기**  
  - 예제:  
    - **1B(10억 개) 파라미터 모델을 fp16(2바이트)로 저장하면 2GB**:
      - 10억 × 2바이트 = 2,000,000,000바이트 = 2,000,000,000 ÷ 1,024^3 ≈ 2GB
    - **7B(70억 개) 파라미터 모델을 16비트(2바이트) 데이터 형식으로 저장하면 14GB**:
      - 70억 × 2바이트 = 14,000,000,000바이트 = 14,000,000,000 ÷ 1,024^3 ≈ 14GB
  - 모델 크기가 증가하면서 메모리 최적화 기술이 중요해짐.

---

### 양자화로 모델 용량 줄이기

- **양자화(Quantization) 개념**  
  - 모델의 용량을 줄이기 위해 **더 적은 비트**로 표현하는 기술.  
  - 예: **fp32 → fp16 변환** 시 용량 절감 가능하지만, 일부 정보 손실 발생.  
  - 핵심 목표: **정보 손실을 최소화하면서 비트 수를 줄이는 것**.  

- **양자화 방식**  
  1. **fp32 → int8 변환**  
     - fp32(32비트) 대신 int8(8비트) 사용하여 모델 크기 축소.  
     - int8은 **256개 숫자(-128 ~ 127)만 표현 가능**, 일부 숫자 손실 문제 발생.  
     <img src="./images/fig_05_02.png" width="800">

  2. **절대 최댓값 활용(absmax 방식)**  
     - **데이터의 최대/최소값을 기준**으로 정규화하여 변환.  
     - 예: 데이터 값이 -17 ~ 15이면, 최대 절댓값 17을 기준으로 int8 범위에 맞춤.  
     <img src="./images/fig_05_03.png" width="800">

     - 단점: **이상치(outlier)** 에 민감하여 일부 값이 왜곡될 수 있음.  
     <img src="./images/fig_05_04.png" width="800">

  3. **블록 단위 양자화(Block-wise Quantization)**  
     - 전체 데이터가 아닌, **소규모 블록(예: 3개씩 묶음)으로 양자화**.  
     - 이상치가 있는 경우, 블록 내에서 상대적인 크기를 조정하여 변환.  
     - 중앙에 모여 있는 일반적인 데이터는 int8 변환, 근단 값은 부동소수점으로 유지

  4. **퀀타일(Quantile) 방식의 양자화**  
     - 기존의 **절대 최댓값(absmax) 방식** 대신, **입력 데이터의 크기 순서(등수)를 기반으로 변환**.  
     - **각 int8 값이 동일한 개수의 fp32 값에 대응되도록 매핑**.  
     <img src="./images/fig_05_05.png" width="800">
     - int8 값의 낭비가 없음 (양 끝값 부족 문제 해결).  
     - 모든 입력 데이터의 순서를 확인해야 하므로 계산량이 많음.  
     - 정렬된 데이터 순서를 저장해야 하므로 추가적인 메모리 사용 필요.  

---

### GPU 메모리 분해하기

- **GPU 메모리에 저장되는 주요 데이터**  
  - **모델 파라미터**  
  - **그래디언트 (Gradient)**  
  - **옵티마이저 상태 (Optimizer State)**  
  - **순전파 상태 (Forward Activation)**  

- **딥러닝 학습 과정과 메모리 사용**  
  - 순전파 → 손실 계산 → 역전파 → 옵티마이저를 통한 모델 업데이트.  
  - 역전파 시, 순전파에서 저장된 값과 그래디언트를 사용.  
  - 옵티마이저는 추가적인 상태 변수를 저장해야 하므로 메모리 사용 증가.  

- **메모리 사용량 계산 (fp16 모델 기준, AdamW 옵티마이저 사용)**  
  - **모델 파라미터:** $ 2N = N \times 2 $ 바이트  
  - **그래디언트:** $ 2N = N \times 2 $ 바이트  
  - **옵티마이저 상태:** $ 4N = N \times 2 \times 2 $ 바이트  
  - **기타:** 순전파 상태(배치 크기, 시퀀스 길이, 잠재상태 크기 등)

- **GPU 메모리 예측 도구**  
  - 허깅페이스에서 **모델 메모리 계산기** 제공:  
    - [모델 메모리 계산기](https://huggingface.co/spaces/hf-accelerate/model-memory-usage)  

  <img src="./images/fig_05_06.png" width="800">

  - EleutherAI의 한국어 모델 **polyglot-ko-1.3b** 예제:  
    - 모델 크기: **2.37GB**  
    - 학습 시 필요한 메모리: **9.49GB**  

**예제 5.1. 메모리 사용량 측정을 위한 함수 구현**
- 코드 실행을 통해 모델을 로드하고 학습하면서 GPU 메모리 사용량을 확인.
- 이를 위해 PyTorch의 `torch.cuda.memory_allocated()` 함수를 사용하여 메모리 사용량을 확인.
- 메모리 사용량을 기가바이트(GiB) 단위로 변환하는 함수 `print_gpu_utilization`을 정의.
- Google Colab에서 예제 5.1을 실행하여 현재 GPU 메모리를 사용하고 있지 않음을 확인 가능.

In [3]:
# PyTorch 라이브러리 임포트
import torch

# GPU 메모리 사용량을 GB 단위로 출력하는 함수
def print_gpu_utilization():
    # GPU 사용 가능 여부 확인
    if torch.cuda.is_available():
        # 현재 할당된 GPU 메모리를 GB 단위로 변환
        used_memory = torch.cuda.memory_allocated() / 1024**3
        print(f"GPU 메모리 사용량: {used_memory:.3f} GB")
    else:
        # GPU를 사용할 수 없는 경우 안내 메시지 출력
        print("런타임 유형을 GPU로 변경하세요")

# 함수 실행하여 현재 GPU 메모리 사용량 확인
print_gpu_utilization()
# 출력 결과
# GPU 메모리 사용량: 0.000 GB

GPU 메모리 사용량: 0.000 GB


**예제 5.2. 모델을 불러오고 GPU 메모리와 데이터 타입 확인**
- `load_model_and_tokenizer` 함수를 정의하여 모델과 토크나이저를 불러옴.
- 이 함수는 **모델 ID (`model_id`)** 와 **효율적인 학습 방식 (`peft` 인자)** 을 입력받음.
- `model_id`를 사용해 `AutoTokenizer` 및 `AutoModelForCausalLM`의 `from_pretrained()` 메서드를 호출하여 모델과 토크나이저를 로드.
- `peft` 인자는 이후 5.3절과 5.4절에서 자세히 다루며, 현재는 기본값 `None`을 사용.
- **예제 5.2 실행 결과**: GPU 메모리 사용량 **2.599GB**.
- 모델의 **파라미터 데이터 타입**: `torch.float16`.
- 이전 모델 메모리 계산기에서 모델 크기를 **2.37GB**로 추정했으며, 이는 비교적 정확한 값임.

In [4]:
# transformers 라이브러리에서 필요한 클래스들을 임포트
from transformers import AutoModelForCausalLM, AutoTokenizer

# 모델과 토크나이저를 로드하는 함수 정의
def load_model_and_tokenizer(model_id, peft=None):
    # 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    
    # peft가 None인 경우 기본 모델 로드
    # torch_dtype="auto"로 설정하여 자동으로 데이터 타입 결정
    # device_map으로 첫 번째 GPU에 모델 할당
    if peft is None:
        model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map={"":0})

    # 현재 GPU 메모리 사용량 출력
    print_gpu_utilization()
    return model, tokenizer

# EleutherAI의 한국어 모델 ID 설정
model_id = "EleutherAI/polyglot-ko-1.3b"

# 모델과 토크나이저 로드 (GPU 메모리 약 2.6GB 사용)
model, tokenizer = load_model_and_tokenizer(model_id) # GPU 메모리 사용량: 2.599 GB

# 모델의 파라미터 데이터 타입 출력 (float16으로 설정됨)
print("모델 파라미터 데이터 타입: ", model.dtype) # torch.float16

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

GPU 메모리 사용량: 2.482 GB
모델 파라미터 데이터 타입:  torch.float16


**예제 5.3. 그레이디언트와 옵티마이저 상태의 메모리 사용량을 계산하는 함수**
- **GPU 메모리에는** 모델, 그래디언트, 옵티마이저 상태, 순전파 상태 등이 저장됨.
- 그래디언트 메모리 사용량을 확인하는 `estimate_memory_of_gradients` 함수 정의.
- 옵티마이저 상태 메모리 사용량을 확인하는 `estimate_memory_of_optimizer` 함수 정의.
- **`estimate_memory_of_gradients` 함수**: 인자로 모델을 받아 그래디언트 값의 **수** (`param.grad.nelement`)와 **데이터 크기** (`param.grad.element_size`)를 곱하여 전체 메모리 사용량 계산.
- **`estimate_memory_of_optimizer` 함수**: 인자로 옵티마이저를 받아 저장된 값의 **수** (`v.nelement`)와 **데이터 크기** (`v.element_size`)를 곱하여 전체 메모리 사용량 계산.

In [5]:
# 필요한 라이브러리 임포트
from transformers import AdamW  # AdamW 옵티마이저 임포트
from torch.utils.data import DataLoader  # 데이터 로더 임포트

def estimate_memory_of_gradients(model):
    """
    모델의 그래디언트가 사용하는 메모리를 계산하는 함수
    Args:
        model: 메모리를 계산할 모델
    Returns:
        total_memory: 그래디언트가 사용하는 총 메모리 (bytes)
    """
    total_memory = 0
    for param in model.parameters():
        if param.grad is not None:
            # 각 파라미터의 그래디언트 크기 * 데이터 타입 크기를 합산
            total_memory += param.grad.nelement() * param.grad.element_size()
    return total_memory

def estimate_memory_of_optimizer(optimizer):
    """
    옵티마이저 상태가 사용하는 메모리를 계산하는 함수
    Args:
        optimizer: 메모리를 계산할 옵티마이저
    Returns:
        total_memory: 옵티마이저 상태가 사용하는 총 메모리 (bytes)
    """
    total_memory = 0
    for state in optimizer.state.values():
        for k, v in state.items():
            if torch.is_tensor(v):
                # 옵티마이저 상태 크기 * 데이터 타입 크기를 합산
                total_memory += v.nelement() * v.element_size()
    return total_memory

**예제 5.4. 모델의 학습 과정에서 메모리 사용량을 확인하는 train_model 정의**
- **`train_model` 함수 정의**: 모델 학습 중 메모리 사용량을 확인하는 기능을 포함.
- 함수의 내용은 **허깅페이스 트랜스포머 Trainer API 없이 학습**할 때 사용했던 코드와 유사함.
- **`training_args.gradient_checkpointing` 설정**:
  - 그래디언트 체크포인팅 기능 사용 여부를 결정.
  - `True` 또는 `False`로 설정 가능하며, 기본값은 `False`(사용하지 않음).
- **`training_args.gradient_accumulation_steps` 설정**:
  - 그래디언트 누적 기능 사용 여부 결정.
  - 누적할 스텝 수를 **1, 2, 4 등으로 설정 가능**(기본값: 1).
- **그래디언트 및 옵티마이저 메모리 사용량 출력**:
  - 학습 과정에서 그래디언트와 옵티마이저가 차지하는 메모리 및 전체 메모리 사용량을 계산하여 출력.

In [6]:
def train_model(model, dataset, training_args):
    # 그래디언트 체크포인팅이 활성화되어 있으면 모델에 체크포인팅 기능을 활성화
    if training_args.gradient_checkpointing:
        model.gradient_checkpointing_enable()

    # 데이터로더 생성 및 옵티마이저 초기화
    train_dataloader = DataLoader(dataset, batch_size=training_args.per_device_train_batch_size)
    optimizer = AdamW(model.parameters())
    model.train()
    gpu_utilization_printed = False

    # 학습 루프 시작
    for step, batch in enumerate(train_dataloader, start=1):
        # 배치를 GPU로 이동
        batch = {k: v.to(model.device) for k, v in batch.items()}

        # 순전파 및 손실 계산
        outputs = model(**batch)
        loss = outputs.loss
        # 그래디언트 누적을 위한 손실값 스케일링
        loss = loss / training_args.gradient_accumulation_steps
        # 역전파 수행
        loss.backward()

        # 그래디언트 누적 스텝마다 옵티마이저 업데이트
        if step % training_args.gradient_accumulation_steps == 0:
            optimizer.step()
            # 메모리 사용량 계산
            gradients_memory = estimate_memory_of_gradients(model)
            optimizer_memory = estimate_memory_of_optimizer(optimizer)
            # GPU 사용량 출력 (1회만)
            if not gpu_utilization_printed:
                print_gpu_utilization()
                gpu_utilization_printed = True
            # 그래디언트 초기화
            optimizer.zero_grad()

    # 최종 메모리 사용량 출력
    print(f"옵티마이저 상태의 메모리 사용량: {optimizer_memory / (1024 ** 3):.3f} GB")
    print(f"그레디언트 메모리 사용량: {gradients_memory / (1024 ** 3):.3f} GB")

**예제 5.5. 랜덤 데이터셋을 생성하는 make_dummy_dataset 정의**
- 모델 학습을 위해 **데이터가 필요**하지만, 학습 과정에서 **메모리 사용량**에 집중하기 위해 랜덤 데이터를 생성.
- **예제 5.5**에서 `make_dummy_dataset` 함수를 정의하여 랜덤 데이터를 생성.
- `make_dummy_dataset` 함수:
  - **텍스트 길이**: 256
  - **데이터 개수**: 64개
  - 생성된 더미 데이터를 **`datasets` 라이브러리의 `Dataset` 형태**로 변환 후 반환.

In [7]:
# numpy와 datasets 라이브러리 임포트
import numpy as np
from datasets import Dataset

def make_dummy_dataset():
    # 시퀀스 길이와 데이터셋 크기 설정
    seq_len, dataset_size = 256, 64
    
    # 입력 ID와 레이블로 구성된 더미 데이터 생성
    # 100~30000 사이의 랜덤한 정수로 구성
    dummy_data = {
        "input_ids": np.random.randint(100, 30000, (dataset_size, seq_len)),
        "labels": np.random.randint(100, 30000, (dataset_size, seq_len)),
    }
    
    # 딕셔너리를 Dataset 객체로 변환
    dataset = Dataset.from_dict(dummy_data)
    # PyTorch 텐서 형식으로 설정
    dataset.set_format("pt")
    return dataset

**예제 5.6. 더이상 사용하지 않는 GPU 메모리를 반환하는 cleanup 함수**
- GPU 메모리의 데이터를 삭제하는 `cleanup` 함수 정의.
- `cleanup` 함수에서 수행하는 작업:
  - **GPU 메모리에 올라간 모델(`model`)과 데이터셋(`dataset`) 변수를 삭제**.
  - **`gc.collect()` 함수 실행**: 사용하지 않는 메모리를 회수하는 **가비지 컬렉션 수행**.
  - **`torch.cuda.empty_cache()` 함수 실행**: 더 이상 사용하지 않는 **GPU 메모리를 반환**.

In [8]:
import gc

# GPU 메모리 정리를 위한 cleanup 함수
def cleanup():
    # 전역 변수에서 model이 있다면 삭제
    if 'model' in globals():
        del globals()['model']
    # 전역 변수에서 dataset이 있다면 삭제  
    if 'dataset' in globals():
        del globals()['dataset']
    # 가비지 컬렉션 수행
    gc.collect()
    # GPU 캐시 메모리 비우기
    torch.cuda.empty_cache()

**예제 5.7. GPU 사용량을 확인하는 gpu_memory_experiment 함수 정의**
- 앞서 정의한 함수들을 종합하여 **배치 크기, 그래디언트 누적, 그래디언트 체크포인팅, PEFT 설정에 따른 GPU 사용량**을 확인하는 `gpu_memory_experiment` 함수 정의.
- `load_model_and_tokenizer` 및 `make_dummy_dataset` 함수를 사용하여 **모델, 토크나이저, 데이터셋 로드**.
- 실험 설정에 따라 **학습에 사용할 인자 설정**.
- `train_model` 함수를 통해 **모델 학습을 진행하면서 GPU 메모리 사용량 측정**.
- 실험 종료 후 **모델과 데이터셋 삭제, 사용하지 않는 GPU 메모리 반환**.

In [9]:
# transformers 라이브러리에서 학습에 필요한 클래스들을 임포트
from transformers import TrainingArguments, Trainer

def gpu_memory_experiment(batch_size,
                          gradient_accumulation_steps=1, # 그래디언트 누적 단계 수
                          gradient_checkpointing=False,  # 그래디언트 체크포인팅 사용 여부
                          model_id="EleutherAI/polyglot-ko-1.3b", # 사용할 모델 ID
                          peft=None): # PEFT 설정
    
    # 현재 실험의 배치 크기 출력
    print(f"배치 사이즈: {batch_size}")
    
    # 모델과 토크나이저 로드
    model, tokenizer = load_model_and_tokenizer(model_id, peft=peft)
    
    # 그래디언트 체크포인팅이나 QLora 사용 시 캐시 비활성화
    if gradient_checkpointing == True or peft == 'qlora':
        model.config.use_cache = False

    # 더미 데이터셋 생성
    dataset = make_dummy_dataset()

    # 학습 인자 설정
    training_args = TrainingArguments(
        per_device_train_batch_size=batch_size,      # GPU당 배치 크기
        gradient_accumulation_steps=gradient_accumulation_steps,  # 그래디언트 누적 단계
        gradient_checkpointing=gradient_checkpointing,  # 그래디언트 체크포인팅 사용 여부
        output_dir="./models",  # 결과 저장 경로
        num_train_epochs=1      # 학습 에포크 수
      )

    try:
        # 모델 학습 시도
        train_model(model, dataset, training_args)
    except RuntimeError as e:
        # CUDA 메모리 부족 에러 처리
        if "CUDA out of memory" in str(e):
            print(e)
        else:
            raise e
    finally:
        # 실험 종료 후 메모리 정리
        del model, dataset
        gc.collect()
        torch.cuda.empty_cache()
        print_gpu_utilization()

**예제 5.8. 배치 사이즈를 변경하며 메모리 사용량 측정**
- 배치 크기를 **4에서 16까지 증가**시키며 **GPU 메모리 사용량 변화 확인**.
- `gpu_memory_experiment` 함수에서 **`batch_size` 값을 변경**하여 메모리 사용량을 분석.
- 실험 실행 전, **`cleanup` 및 `print_gpu_utilization` 함수**를 사용하여 **불필요한 GPU 데이터 삭제 및 메모리 반환 확인**.

In [10]:
# GPU 메모리 정리
cleanup()
# 현재 GPU 메모리 사용량 출력
print_gpu_utilization()

# 배치 사이즈를 4, 8, 16으로 증가시키며 실험
for batch_size in [4, 8, 16]:
    # 각 배치 사이즈별 GPU 메모리 실험 수행
    gpu_memory_experiment(batch_size)
    
    # 실험 후 GPU 캐시 메모리 비우기
    torch.cuda.empty_cache()

GPU 메모리 사용량: 0.000 GB
배치 사이즈: 4


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

GPU 메모리 사용량: 2.482 GB
GPU 메모리 사용량: 10.375 GB
옵티마이저 상태의 메모리 사용량: 4.961 GB
그레디언트 메모리 사용량: 2.481 GB
GPU 메모리 사용량: 0.016 GB
배치 사이즈: 8


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

GPU 메모리 사용량: 2.498 GB
GPU 메모리 사용량: 10.808 GB
옵티마이저 상태의 메모리 사용량: 4.961 GB
그레디언트 메모리 사용량: 2.481 GB
GPU 메모리 사용량: 0.016 GB
배치 사이즈: 16


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

GPU 메모리 사용량: 2.498 GB
GPU 메모리 사용량: 11.672 GB
옵티마이저 상태의 메모리 사용량: 4.961 GB
그레디언트 메모리 사용량: 2.481 GB
GPU 메모리 사용량: 0.016 GB


- **배치 크기 증가**에 따른 **GPU 메모리 사용량 변화 확인**.
- **표 5.1**:
  - **배치 크기 증가**에도 **모델, 그래디언트, 옵티마이저 상태의 메모리 사용량은 동일**.
    - **그래디언트(Gradient)** 는 각 파라미터의 손실 변화율을 저장하며, 파라미터 개수와 데이터 타입에 따라 결정되므로 배치 크기에 관계없이 메모리 사용량이 일정합니다.  
    - **옵티마이저 상태(Optimizer State)** 는 각 파라미터의 모멘텀과 적응적 학습률 정보를 저장하므로, 메모리 사용량은 파라미터 수에 따라 결정되며 배치 크기와 무관합니다.
  - **총 메모리 사용량 증가**는 **순전파 상태 계산에 필요한 메모리 증가 때문**.
  - 배치 크기 **16에서 OOM (Out of Memory) 발생**.

  <img src="./images/tbl_05_01.png" width="800">
  
- **GPU 메모리 사용량을 분석하기 위한 함수 정의 및 실험 수행**.
- 이제 **GPU를 효율적으로 활용하는 그래디언트 누적 및 체크포인팅 기법 학습** 예정.

---

## 단일 GPU 효율적으로 활용하기

- **GPU 메모리의 한계**  
  - 모델 크기 및 학습 배치 크기가 GPU 메모리에 의해 제한됨.  
- **효율적인 GPU 메모리 활용법**  
  1. **그래디언트 누적 (Gradient Accumulation)**  
     - 각 배치마다 모델을 업데이트하지 않고 여러 배치의 학습 데이터를 연산 후 업데이트.  
     - 마치 더 큰 배치 크기를 사용하는 것과 같은 효과를 제공.  
  2. **그래디언트 체크포인팅 (Gradient Checkpointing)**  
     - 순전파 계산 결과를 모두 저장하지 않고 일부만 저장.  
     - 역전파 시 필요한 부분만 다시 계산하여 GPU 메모리 사용량 절감.  
- **효과**  
  - 두 방법 모두 모델 학습 시 배치 크기를 키울 수 있어 학습 속도를 빠르고 안정적으로 개선.

---  

### 그레이디언트 누적

- **배치 크기 증가의 효과와 문제점**  
  - 배치 크기를 키우면 학습 속도가 빨라지고 성능이 향상됨.  
  - 하지만 메모리 사용량이 증가하여 OOM(Out of Memory) 오류 발생 가능.  

- **그래디언트 누적의 개념**  
  - 제한된 GPU 메모리 내에서 배치 크기를 키우는 것과 동일한 효과를 제공.  
  - 여러 스텝의 그래디언트를 누적한 후 한 번에 모델을 업데이트하는 방식.  

**예제 5.9 예제 5.4에서 그래디언트 누적과 관련된 부분 설명**

  - `train_model` 함수에서 **gradient_accumulation_steps** 설정 가능.  
  - 예를 들어 **gradient_accumulation_steps = 4**로 설정하면:  
    1. 손실을 4로 나누어 역전파 수행.  
    2. 4번의 스텝마다 모델 업데이트 진행.  
  - 결과적으로 배치 크기가 4배 증가한 것과 같은 효과를 얻음.

In [11]:
def train_model(model, dataset, training_args):
    # 그래디언트 체크포인팅이 활성화된 경우 모델에 적용
    if training_args.gradient_checkpointing:
        model.gradient_checkpointing_enable()

    # 데이터로더 설정 및 옵티마이저 초기화
    train_dataloader = DataLoader(dataset, batch_size=training_args.per_device_train_batch_size)
    optimizer = AdamW(model.parameters())
    model.train()
    gpu_utilization_printed = False

    # 배치 단위로 학습 진행
    for step, batch in enumerate(train_dataloader, start=1):
        # 배치 데이터를 GPU로 이동
        batch = {k: v.to(model.device) for k, v in batch.items()}

        # 모델 순전파 및 손실 계산
        outputs = model(**batch)
        loss = outputs.loss
        # 그래디언트 누적을 위해 손실을 누적 스텝 수로 나눔
        loss = loss / training_args.gradient_accumulation_steps
        loss.backward()

        # 그래디언트 누적 스텝마다 모델 업데이트
        if step % training_args.gradient_accumulation_steps == 0:
            optimizer.step()
            # 메모리 사용량 측정
            gradients_memory = estimate_memory_of_gradients(model)
            optimizer_memory = estimate_memory_of_optimizer(optimizer)
            # GPU 사용량 한 번만 출력
            if not gpu_utilization_printed:
                print_gpu_utilization()
                gpu_utilization_printed = True
            optimizer.zero_grad()

**예제 5.10. 그레이디언트 누적을 적용했을 때 메모리 사용량**

- **OOM(Out of Memory) 발생 사례**  
  - 배치 크기를 **16**으로 설정했을 때 **OOM 오류 발생**.  

- **그래디언트 누적 활용 결과**  
  - 배치 크기를 **4**로 설정하고,  
  - **gradient_accumulation_steps = 4**로 설정하여 실행.  
  - 결과적으로 **GPU 메모리 사용량 10.586GB**로 성공적으로 실행됨.  
  - 배치 크기 **16일 때와 동일한 효과**를 얻음.  
  - **GPU 메모리 절약**하면서도 큰 배치 크기의 효과를 얻을 수 있음.  
  - 하지만 추가적인 **순전파 및 역전파 연산이 필요**하여 학습 시간이 증가함.

In [ ]:
cleanup()
print_gpu_utilization()

gpu_memory_experiment(batch_size=16, gradient_accumulation_steps=4)

torch.cuda.empty_cache()

GPU 메모리 사용량: 0.016 GB
배치 사이즈: 4


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

GPU 메모리 사용량: 2.498 GB
GPU 메모리 사용량: 10.375 GB
GPU 메모리 사용량: 0.016 GB


---

### 그레디언트 체크포인팅

- **기본적인 그래디언트 저장 방식**  
  - 모델 업데이트를 위해 **순전파 결과를 저장**해야 함.  
  - 기본 방식은 **모든 결과를 저장**, 하지만 **GPU 메모리를 많이 차지**함.  

- **그래디언트 체크포인팅의 개념**  
  - **필요한 데이터만 선택적으로 저장**하여 메모리 절약.  
  - 역전파 시 사용이 끝난 데이터는 **즉시 삭제**.  
  - **장점** : GPU 메모리 사용량 절감.  
  - **단점** : 하지만 일부 순전파 계산을 다시 수행해야 하므로 **학습 속도가 다소 느려질 수 있음**.

  <img src="./images/fig_05_07.png" width="800">

- **메모리 절약을 위한 최적의 그래디언트 체크포인팅 방법**
  - **필요한 최소 데이터만 저장**하고, 나머지는 필요할 때 다시 계산.  
  - 순전파 과정에서 **중간 데이터를 삭제**하며 진행하여 **메모리 절약 극대화**.  
  - 마지막 데이터만 남긴 상태에서 손실을 계산. 
  - 역전파 계산 시 **순전파 결과가 필요하면 처음부터 다시 계산**.  
  - **장점**  
    - GPU 메모리를 매우 효율적으로 활용 가능.
    - 대규모 모델에서도 학습 가능하도록 메모리 요구량 감소.
  - **단점**  
    - 역전파 시 순전파를 반복적으로 다시 계산해야 하므로 학습 속도 감소.
  - **그림 5.8 해석**  
    - **(a)**: 순전파 중 거의 모든 중간 데이터를 삭제하여 메모리 절약.  
    - **(b)**: 일부 중요한 중간 데이터를 체크포인트로 저장하여 역전파 시 재계산 부담 완화.  

  <img src="./images/fig_05_08.png" width="800">

- **그래디언트 체크포인팅 (Gradient Checkpointing) 방식 최적화**  
  - 순전파의 전체 결과를 저장하는 대신, **중간 노드(체크포인트)만 저장**하여 메모리 사용 절감.  
  - 역전파 계산 시 체크포인트를 활용하여 **다시 계산할 양을 최소화**.  
  - **장점**  
    - **GPU 메모리 사용량 절감**으로 대규모 모델 학습 가능.  
    - **순전파 재계산 양을 줄여 학습 속도 최적화**.  
  - **단점**  
    - 여전히 **일부 순전파를 다시 계산해야 하므로 속도 저하 가능**.
  - **작동 방식 (그림 5.9 해석)**  
    - **(a)**: 순전파 중 특정 노드(체크포인트)만 저장.  
    - **(b)**: 순전파 완료 후, 체크포인트와 마지막 노드만 유지.  
    - **(c)**: 역전파 시, 저장된 체크포인트 이후의 순전파를 다시 계산하여 필요한 값 확보.  

  <img src="./images/fig_05_09.png" width="800">

**예제 5.11. 그레이디언트 체크포인팅 사용 시 메모리 사용량**

  - **배치 크기 16**일 때 **OOM 오류 발생**.  
  - 그래디언트 체크포인팅 적용 후 **GPU 메모리 사용량 10.29GB로 감소**, 학습 성공.  
  - **GPU 메모리를 효율적으로 사용하여 동일한 환경에서 더 큰 배치 크기로 학습 가능**.

In [13]:
cleanup()
print_gpu_utilization()

gpu_memory_experiment(batch_size=16, gradient_checkpointing=True)

torch.cuda.empty_cache()

GPU 메모리 사용량: 0.016 GB
배치 사이즈: 16


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

GPU 메모리 사용량: 2.498 GB
GPU 메모리 사용량: 10.172 GB
GPU 메모리 사용량: 0.016 GB


- **그레디언트 누적(Gradient Accumulation)과 그레디언트 체크포인팅(Gradient Checkpointing)**  
  - GPU를 1개만 사용하는 학습에서도 활용 가능한 GPU 효율화 기법  
  - GPU 메모리 절약에 도움  

- **GPU 메모리 부족 해결 방법**  
  - 가장 직접적인 해결책: 더 많은 GPU 사용하여 총 메모리 용량 증가  
  - **분산 학습(Distributed Training)**  
    - 2개 이상의 GPU를 활용하여 모델 학습  
    - GPU 리소스를 보다 효율적으로 사용 가능    

---

### 데이터 병렬화에서 중복 저장 줄이기(ZeRO)

- **데이터 병렬화의 비효율성**  
  - 각 GPU가 동일한 모델을 저장해야 하므로 **메모리 낭비 발생**  
  - 모델이 클수록 GPU마다 하나의 전체 모델을 저장하는 것은 더욱 비효율적  

- **ZeRO (Zero Redundancy Optimizer) 소개**  
  - **마이크로소프트에서 2019년 개발**  
  - 모델을 하나의 GPU에 올리는 대신 **여러 GPU에 나누어 분산 저장**  
  - 각 GPU는 **자신이 맡은 모델 부분만 연산 수행**  
  - 불필요한 중복 저장을 줄여 **메모리 사용을 최적화하면서 속도 유지**  

- **ZeRO의 작동 방식**  
  - 기존 방식(베이스라인)에서는 **각 GPU마다 모델 전체의 파라미터, 그래디언트, 옵티마이저 상태를 저장**  
  - **ZeRO는 모델을 부분적으로 GPU에 저장**하고, **필요한 순간에만 파라미터를 복사하여 연산 수행**  
  - 이를 통해 **메모리를 더욱 효율적으로 활용 가능**

  <img src="./images/fig_05_14.png" width="800">

- **ZeRO의 메모리 절감 효과**  
  - GPU를 여러개 사용할수록 **각 GPU의 메모리 사용량 감소**  
  - **64개의 GPU**로 **7.5B(75억) 파라미터 모델** 학습 시  
    - **ZeRO 사용**: **GPU당 1.9GB 메모리** 필요  
    - **기존 방식**: **총 120GB 메모리** 필요  
    - **메모리 사용량이 기존 대비 1/60 수준으로 감소**  

- **ZeRO의 활용 환경**  
  - 허깅페이스(Hugging Face) **트랜스포머에서 쉽게 적용 가능**  
  - 본 책에서는 **GPU 1개만 사용 가능한 환경**(구글 코랩) 기준이므로 실습 진행 없음  

- **ZeRO 사용법 참고 자료**  
  - **Hugging Face Accelerate의 DeepSpeed 사용 가이드**에서 확인 가능  
  - 참고 링크: [Accelerate DeepSpeed Guide](https://huggingface.co/docs/accelerate/usage_guides/deepspeed)

---  

>**Exo**
>
>**Exo**는 다양한 기기를 하나의 강력한 GPU처럼 활용하여 대규모 언어 모델(LLM)을 실행할 수 있게 해주는 분산형 소프트웨어 솔루션입니다. 이를 통해 고성능 GPU 없이도 일반적인 컴퓨터, 스마트폰, Raspberry Pi 등으로 구성된 클러스터에서 대규모 AI 모델을 실행할 수 있습니다. 
>
>**주요 특징:**
>
>- **동적 모델 파티셔닝**: Exo는 네트워크 구조와 각 디바이스의 리소스를 분석하여 모델을 최적으로 분할합니다. 이를 통해 단일 디바이스의 한계를 넘어서는 대규모 모델 실행이 가능합니다. 
>
>- **자동 디바이스 검색**: 네트워크에 연결된 디바이스를 자동으로 발견하여, 수동 설정 없이 클러스터를 형성합니다. 
>
>- **ChatGPT 호환 API**: 기존 애플리케이션에서 최소한의 코드 변경으로 Exo를 통합할 수 있도록 ChatGPT와 호환되는 API를 제공합니다. 
>
>- **디바이스 평등**: 마스터-워커 아키텍처 대신 P2P 방식을 채택하여 모든 디바이스가 동등하게 모델 실행에 참여할 수 있습니다. 
>
>**작동 방식:**
>
>1. **모델 분할**: Exo는 "ring memory weighted partitioning"과 같은 전략을 사용하여 각 디바이스의 메모리 용량에 따라 모델 레이어를 분배합니다. 
>
>2. **리소스 활용**: iPhone, iPad, Android, Mac, NVIDIA GPU, Raspberry Pi 등 다양한 디바이스를 통합하여 하나의 강력한 GPU처럼 활용합니다. 
>
>3. **모델 지원**: LLaMA, Mistral, LlaVA, Qwen, Deepseek 등 다양한 모델을 지원합니다. 
>
>Exo는 고성능 GPU가 없는 사용자도 다양한 기기를 활용하여 대규모 AI 모델을 실행할 수 있게 해주는 혁신적인 접근 방식을 제공합니다. 이는 기존의 분산 처리 기술과 유사하지만, 더 다양한 디바이스를 활용할 수 있다는 점에서 차별화됩니다.
>
>Exo의 실제 작동 방식과 설정 방법에 대한 시각적 이해를 돕기 위해 아래 영상을 참고하실 수 있습니다.
>
>[![Exo 소개 영상](https://img.youtube.com/vi/Ju0ndy2kwlw/0.jpg)](https://youtu.be/Ju0ndy2kwlw?si=n_6_mk9L8MH0ue9q)

---
 

## 효율적인 학습 방법(PEFT) : LoRA

- **LLM(대형 언어 모델)의 학습 문제점**  
  - 모델 크기가 커지면서 **모든 파라미터를 학습하는 전체 미세 조정(full fine-tuning)** 이 어려워짐  
  - 개인 및 조직에서 **여러 GPU를 사용하기 어려운 환경**이 많음  

- **해결책: PEFT (Parameter Efficient Fine-Tuning) 연구**  
  - **일부 파라미터만 학습하여 효율적으로 모델을 조정하는 방법** 연구 활발  

- **LoRA (Low-Rank Adaptation) 개요**  
  - **오픈소스 LLM 학습에서 가장 주목받는 방법**  
  - 기존 모델에 **일부 파라미터를 추가한 후, 해당 부분만 학습**하여 효율성 향상  
---  

### 모델 파라미터의 일부만 재구성해 학습하는 LoRA

- **LoRA의 핵심 개념**  
  - **LLM 시대에서 가장 사랑받는 PEFT 방법 중 하나**  
  - 기존 모델의 **파라미터를 재구성(reparameterization)하여 일부만 학습**  
  - 이를 통해 **GPU 메모리 사용량을 절감**  

- **LoRA의 파라미터 재구성 방식**  
  - 기존 파라미터를 직접 수정하는 것이 아니라, **더 작은 2개의 행렬 곱으로 표현**  
  - **이 작은 행렬만 수정하여 학습 진행**  

- **LoRA의 파라미터 수 (그림 5.15)**  
  - 딥러닝 모델이 **차원 $ d $ 의 입력 $ X $ 와 파라미터 $ W $ 를 곱해 출력 $ h $ 생성**  
  - 파라미터 $ W $ 가 $ d \times d $ 차원일 때, $ d = 100 $ 이라면  
  - 전체 파라미터 $ W $ 를 학습하면 **10,000개의 파라미터 학습 필요**  

  <img src="./images/fig_05_15.png" width="800">

- **LoRA의 장점**  
  - **더 적은 파라미터만 학습하여 효율적으로 모델 튜닝 가능**  
  - **GPU 메모리 사용량 절감**과 동시에 **학습 속도 개선**

- **LoRA의 파라미터 재구성 방식 (그림 5.16)**  
  - 기존 파라미터 $ W $ 를 직접 학습하는 대신 **$ W $ 를 고정하고, 새로운 두 개의 작은 행렬 $ A $ 와 $ B $ 를 학습**  
  - 행렬 $ A $ 는 $ (d, r) $ 차원, 행렬 $ B $ 는 $ (r, d) $ 차원이며, **이 두 행렬의 곱이 $ W $ 를 보정**  
  - **$ r $ 값을 작게 설정하면 학습해야 할 파라미터 수가 대폭 감소**  
  - 예를 들어 $ r = 4 $ 일 경우,  
    - 기존 $ W $ 를 학습하면 **10,000개 파라미터 필요 (100 × 100)**  
    - LoRA에서는 $ A $ 와 $ B $ 를 학습하면 **800개 파라미터만 필요 (100 × 4 + 4 × 100)**  
    - **결과적으로 전체 학습 파라미터 수를 8% 수준까지 줄일 수 있음**  
    - 일반적으로 $ r $ 값을 더 작게 설정하면 **1% 미만의 파라미터만 학습하는 경우도 존재**  
    - 행렬 $ A $ 와 $ B $ 를 추가하면 **기존 모델의 파라미터 수는 소폭 증가**하지만,  

  <img src="./images/fig_05_16.png" width="800">

- **LoRA vs. 전체 미세 조정 (Full Fine-Tuning) 비교 (그림 5.17)**  
  - GPU 메모리를 차지하는 주요 요소는 모델 파라미터뿐만 아니라 그래디언트와 옵티마이저 상태도 포함됨
  - LoRA는 학습하는 파라미터 수가 줄어들어 그래디언트 및 옵티마이저 상태 저장 공간이 감소 → 전체 메모리 사용량 절감
  - **LoRA 적용 시, 추가된 행렬이 시각적으로 작게 표현됨**  
  - 학습할 파라미터 수가 줄어들면 **업데이트할 옵티마이저 상태 데이터도 줄어들어 추가적인 메모리 절감 효과 발생**

  <img src="./images/fig_05_17.png" width="800">

---

### LoRA 설정 살펴보기

- **차원 $ r $ 설정**   
  - 행렬 $ W $에 더할 행렬 $ A $와 $ B $를 만들 때 차원 $ r $을 결정해야 함.  
  - $ r $을 작게 설정하면 학습시켜야 하는 파라미터 수가 줄어들어 GPU 메모리 사용량 감소.  
  - 하지만 $ r $이 너무 작으면 모델의 학습 용량(capacity)도 줄어 학습 데이터의 패턴을 충분히 학습하지 못할 위험 있음.  
  - 따라서 실험을 통해 적절한 $ r $ 값을 찾아야 함.  
- **알파 ($ \alpha $) 값 설정**  
  - 추가된 파라미터를 기존 파라미터에 얼마나 반영할지 결정하는 역할.  
  - 행렬 $ A $와 $ B $를 추가했을 때, 곱한 부분을 기존 파라미터 $ W $와 동일하게 더할 수도 있지만, 행렬 $ A $와 $ B $를 더 중요하게 고려할 수도 있음.  
  - LoRA에서는 행렬 $ A $와 $ B $를 ($ \alpha / r $)만큼의 비중으로 기존 파라미터 $ W $에 더해줌.  
  - 예시:  
    - $ \alpha = 16 $, $ r = 8 $일 때, $ A $와 $ B $에 $ 2 (=16/8) $를 곱해 기존 파라미터에 더함.  
    - 즉, $ \alpha $가 클수록 새롭게 학습한 파라미터의 중요성을 크게 고려하는 것.  
  - 학습 데이터에 따라 적절한 $ \alpha $ 값이 달라지므로 실험을 통해 적절한 값을 설정해야 함.  

  <img src="./images/fig_05_18.png" width="400">

- **어떤 파라미터를 재구성할지 결정해야 함**  
  - 일반적으로 셀프 어텐션 연산의 주요 가중치(query, key, value)와 피드 포워드 층의 가중치와 같은 **선형 연산의 가중치**를 재구성.  
- **LoRA 적용 범위 선택**  
  - 특정 가중치에만 LoRA를 적용할 수도 있고, 전체 선형 층에 적용할 수도 있음.  
  - 일반적으로 **전체 선형 층에 LoRA를 적용하는 경우 성능이 가장 좋다고 알려짐**.  
  - 하지만 최적의 선택을 위해 **실험을 통해 적절히 결정해야 함**.  

---

### 코드로 LoRA 학습 사용하기

- **LoRA 학습 개요**
  - GPU 메모리 사용량을 줄이면서도 미세 조정과 유사한 성능 제공
  - 허깅페이스의 `peft` 라이브러리를 통해 쉽게 활용 가능

**예제 5.12. 모델을 불러오면서 LoRA 적용하기**
  1. `load_model_and_tokenizer` 함수에서 `peft` 인자를 `'lora'`로 설정하면 LoRA 적용 가능
  2. `peft` 라이브러리의 `LoraConfig` 클래스로 LoRA 설정 정의:
     - `r` 값: 8
     - `alpha` 값: 32
     - `target_modules`: `"query_key_value"`
  3. 모델에 `lora_config` 적용:
     - `get_peft_model` 함수 호출로 LoRA 적용
     - `print_trainable_parameters()` 메서드로 학습 파라미터 수와 비율 확인

In [14]:
# transformers 라이브러리에서 필요한 모델과 토크나이저 임포트
from transformers import AutoModelForCausalLM, AutoTokenizer
# peft 라이브러리에서 LoRA 관련 기능 임포트  
from peft import LoraConfig, get_peft_model

def load_model_and_tokenizer(model_id, peft=None):
    # 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    # peft가 None이면 기본 모델만 로드
    if peft is None:
        model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map={"":0})

    # peft가 'lora'이면 LoRA 설정 적용
    elif peft == 'lora':
        # 기본 모델 로드
        model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map={"":0})
        # LoRA 설정
        lora_config = LoraConfig(
                    r=8,                            # LoRA 랭크
                    lora_alpha=32,                  # LoRA 스케일링 파라미터
                    target_modules=["query_key_value"], # LoRA 적용할 모듈
                    lora_dropout=0.05,              # LoRA 드롭아웃 비율
                    bias="none",                    # 바이어스 학습 여부
                    task_type="CAUSAL_LM"          # 태스크 타입
                )

        # 모델에 LoRA 적용
        model = get_peft_model(model, lora_config)
        # 학습 가능한 파라미터 정보 출력
        model.print_trainable_parameters()

    # GPU 메모리 사용량 출력
    print_gpu_utilization()
    return model, tokenizer

**예제 5.13. LoRA를 적용했을 때 GPU 메모리 사용량 확인**

- **LoRA 적용 효과**
  - 학습 가능한 파라미터 수가 전체 대비 **0.117%로 감소**
  - 전체 모델을 불러올 때의 메모리 사용량: **2.602GB** (기존과 거의 동일)

- **메모리 사용량 감소**
  - 옵티마이저 상태 메모리 사용량 감소: **0.006GB**
  - 그래디언트 메모리 사용량 감소: **0.003GB**
  - 전체 파라미터를 학습하는 것이 아니라 **일부(0.117%)만 학습**하기 때문

In [15]:
# GPU 메모리 정리
cleanup()
# 현재 GPU 메모리 사용량 출력
print_gpu_utilization()

# LoRA를 적용한 GPU 메모리 사용량 실험 (배치 크기 16)
gpu_memory_experiment(batch_size=16, peft='lora')

# GPU 캐시 메모리 비우기
torch.cuda.empty_cache()

GPU 메모리 사용량: 0.016 GB
배치 사이즈: 16


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

trainable params: 1,572,864 || all params: 1,333,383,168 || trainable%: 0.11796039111242178
GPU 메모리 사용량: 2.500 GB
GPU 메모리 사용량: 4.240 GB
GPU 메모리 사용량: 0.016 GB


- **LoRA의 장점**
  - 모델의 **일부만 학습**하여 **GPU 메모리 사용량 감소**
  - 효율적인 모델 학습 가능
  - **전체 미세 조정과 거의 동일한 성능** 달성 가능

- **LoRA의 활용**
  - 높은 효율성과 성능 유지로 **많이 사용됨**
  - **QLoRA**: LoRA에 **모델 양자화**를 추가한 방식으로 더 최적화 가능

---

## 효율적인 학습 방법(PEFT) : QLoRA

- **발표 및 배경**  
  - 2023년 5월 워싱턴대학교의 Tim Dettmers와 Artidoro Pagnoni 팀이 발표함.  
  - 논문 제목: *"QLoRA: Efficient Finetuning of Quantized LLMs"* ([논문 링크](https://arxiv.org/pdf/2305.14314.pdf))  

- **핵심 개념**  
  - LoRA에 **양자화(Quantization)** 기법을 추가하여 메모리 효율을 높인 학습 방법.  
  - 양자화란 기존 데이터를 더 적은 메모리로 변환하여 사용하는 방식.  

- **QLoRA와 LoRA의 비교**  
  - **LoRA**: 모델을 16비트 형식으로 저장.  
  - **QLoRA**: 모델을 **4비트 형식**으로 저장하여 메모리 사용량을 대폭 감소시킴.  

- **메모리 관리 및 안정성**  
  - CPU 메모리와 GPU 메모리를 함께 사용하여 학습 효율성을 높임.  
  - 학습 도중 발생할 수 있는 **OOM (Out Of Memory) 오류**를 방지함.  
  - 이를 위해 **Paged Optimizer** 기능을 활용하여 안정적인 학습 가능.  

    <img src="./images/fig_05_19.png" width="800">

---

### 4비트 양자화와 2차 양자화

- **양자화의 핵심 과제**  
  - 기존 데이터의 정보를 최대한 유지하면서 더 적은 비트를 사용하는 데이터 형식으로 변환함.  
  - 특히 변환 대상 데이터가 적은 비트 수를 사용하면 메모리 사용량이 절감됨.  

- **데이터 순서와 매핑 방법**  
  - 기존에는 데이터 순서대로 새로운 데이터 형식에 매핑했으나, 이 방법은 데이터 정렬 및 순서 저장에 메모리를 사용해야 하는 단점이 있음.  
  - 해결 방법: 데이터의 분포를 알고 있으면 더 적은 연산과 메모리로 빠르게 데이터의 순서를 정할 수 있음.  

- **정규 분포에 따른 경계값 계산**  
  - 입력 데이터가 **정규 분포**를 따르면 데이터 분포에 따라 경계값을 계산하여 데이터를 균등하게 분할할 수 있음.  
  - 예시:  
    - 입력 데이터를 정규 분포에 따라 **10등분**할 경우, 경계값을 찾아 각 영역으로 데이터 분할 가능.  
    - 데이터가 **4비트(16개의 값)** 를 사용할 경우, 정규 분포를 **16등분**하여 경계값을 찾아 데이터를 0~15까지 배정.  

    <img src="./images/fig_05_20.png" width="450">

- **정규 분포 경계값 구하는 방법 (Quantile Normal 함수 사용)**  
  - `quantile_normal(p)` 함수 사용 시, 정규 분포에서 특정 퍼센트(`p%`)의 데이터가 해당 값 이하에 위치함.  
  - 예시:  
    - `quantile_normal(0.5)` → 결과: 0 (정규 분포의 중앙값)  
    - `quantile_normal(0.6)` → 결과: 0.2533 (60%의 데이터가 이 값보다 작음)  
    - 두 결과의 차이인 0.2533이 데이터의 상위 10% 영역을 의미  

      ```python

      from scipy.stats import norm

      def quantile_normal(p):
          return norm.ppf(p)

      print(quantile_normal(0.5)) # 0
      print(quantile_normal(0.6)) # 0.2533471031357996

      ```

- **정규 분포와 모델 파라미터**  
  - 학습된 모델의 파라미터는 일반적으로 **정규 분포**에 가까움.  
  - 실제로 **LLaMa 모델**의 경우, 약 **92.5%** 의 파라미터가 정규 분포를 따름.  
  - 이를 활용하면 모델 성능을 거의 유지하면서도 빠른 양자화가 가능함.  

- **NF4 (Normal Float 4-bit) 양자화**  
  - QLoRA에서는 4비트 양자화를 수행하기 위해 **NF4(4비트 부동소수점 데이터 형식)** 사용.  
  - NF4는 정규 분포를 따르는 입력 데이터에 적합하도록 설계됨.  
  - 모델의 **64개 파라미터**를 하나의 **블록**으로 묶어 양자화 수행.  
    - 전체 모델의 파라미터를 한 번에 평균과 표준편차로 정규화하는 것이 아니라, 64개씩 묶어서 하나의 블록으로 나누고, 각 블록마다 개별적으로 평균과 표준편차를 계산해 정규화.
    - 블록 단위로 정규화하면 각 블록의 값의 특성을 유지하면서도 압축이 가능
  - 각 블록당 **1개의 상수(정규 분포의 표준편차)** 를 저장해야 함.  

>**Tips**
>
>💾 **NF4 양자화(Quantization)**
>1. **64개 파라미터**의 **표준편차 $\sigma$** 를 계산합니다.  
>2. 각 파라미터를 표준편차로 나누어 정규화합니다: $  x_{scaled} = \frac{x}{\sigma}  $
>3. 정규화된 값을 **NF4(4비트)** 범위인 $-1.0$부터 $1.0$ 사이의 **16개 실수값** 중 **가장 가까운 값**으로 매핑합니다:   $  x_{quant} \in \{-1.0, -0.9375, -0.875, \dots, 0.9375, 1.0\}  $
>- 이 **16개의 값**은 **정규 분포의 분위수(quantile)** 에 맞게 **비선형적으로 분포**되어 있습니다.  
>- 이로 인해 **정보 손실을 최소화**할 수 있습니다.
>4. **매핑된 실수값**은 **저장 시** **0부터 15까지의 정수값**으로 변환됩니다.
>
>🔁 **NF4 복원(Dequantization)**
>1. 메모리에 저장된 **4비트 정수값**을 **NF4의 실수값**으로 **Look-up Table (LUT)** 을 사용해 변환합니다.  
>2. 해당 블록의 **표준편차 $\sigma$** 를 곱해 원래 스케일로 복원합니다:  $  x_{original} = x_{quant} \times \sigma  $
>3. **32비트 양자화 상수($\sigma$)** 는 **블록별로 하나만** 저장됩니다.  
>- 각 블록은 **64개의 파라미터**를 동일한 $\sigma$로 복원하므로, 메모리 사용량이 크게 절약됩니다.

- **2차 양자화 (Double Quantization)**  
  - 4비트 양자화 이후 한 단계 더 나아가 **2차 양자화**를 적용함.  
  - 목적: NF4 양자화 과정에서 생성된 **32비트 상수**도 효율적으로 저장.    
  - **256개의 양자화 상수**를 다시 하나의 블록으로 묶어 **8비트 양자화** 수행.  
  - 기존 방식: 256개의 양자화 상수를 **32비트 데이터**로 각각 저장해야 함.  
  - **2차 양자화 적용 후:**  
    - **256개의 32비트 데이터** 대신,  
    - **8비트 데이터 256개** + **32비트 데이터 1개**로 저장하여 메모리 절약.  

    <img src="./images/fig_05_21.png" width="800">


>**Tips**  
>
>**1. 기본 개념: 2차 양자화란?**
>- **NF4 4비트 양자화**를 적용하면 **64개 파라미터당 하나의 32비트 양자화 상수**를 저장해야 함.  
>- 하지만, **모델이 클수록 양자화 상수(표준편차) 자체가 차지하는 메모리가 커짐.**  
>- 따라서, **256개의 양자화 상수를 하나의 블록으로 묶어, 8비트 양자화**를 수행하여 추가적인 메모리 절약을 유도함.  
>- 이를 **"2차 양자화(Double Quantization)"** 라고 부름.  
>
>**2. 수식 설명**
>**기존 방식의 메모리 사용량**
>- **각 블록(64개 파라미터)에 대해 1개의 32비트 양자화 상수 ($\sigma$)를 저장해야 함.**  
>- 따라서 **256개의 블록**이라면, **각 블록당 32비트**이므로: $\text{총 메모리 사용량} = 32 \times 256 $ = **8192비트 (1024바이트)**
>
>**3. 2차 양자화 적용 에 따른 메모리 절약**
>- **256개의 32비트 데이터를 줄이기 위해,**  
>  1. 256개의 $\sigma$를 하나의 블록으로 묶음.  
>  2. 이를 **8비트 정밀도**로 다시 양자화.  
>  3. 즉, 256개의 양자화 상수를 **8비트 정수 값**으로 변환하고,  
>  4. 해당 **양자화 상수를 복원하기 위한 32비트 데이터 1개**만 저장. $\text{메모리 사용량} = 8 \times 256 + 32$ = **2064비트 (258바이트)**
>
>✔ **기존 방식 대비 약 75% 메모리 절약!** 🚀

---

### 페이지 옵티마이저

- **목적:** 그레이디언트 체크포인팅(Gradient Checkpointing) 과정에서 발생할 수 있는 **OOM (Out Of Memory)** 오류 방지  
- **메모리 사용의 차이:**  
  - **순전파 단계:** 3개의 노드 데이터만 저장 → 메모리 사용량이 적음  
  - **역전파 단계:** 5개의 노드 데이터를 저장 → 메모리 사용량이 증가하여 OOM 오류 발생 가능  

<img src="./images/fig_05_22.png" width="800">

- **페이지 옵티마이저의 개념**  
  - GPU가 CPU 메모리(RAM)를 공유하여 사용할 수 있게 해주는 기술.  
  - NVIDIA의 **통합 메모리(Unified Memory)** 는 컴퓨터의 가상 메모리 시스템과 유사한 방식으로 GPU 메모리 관리에 적용됨.  

- **작동 원리**  
  - GPU와 CPU가 메모리를 공유하여 더 많은 메모리를 사용할 수 있도록 함.  
  - 메모리가 가득 차면 일부 데이터를 디스크로 이동시키고 필요할 때 다시 불러오는 **페이징(Paging)** 과정이 이루어짐.  
  - GPU 메모리에 데이터를 모두 담을 수 없으면 일부 데이터를 CPU 메모리에 보관하고, 필요 시 해당 데이터를 CPU 메모리에서 GPU 메모리로 전송함.  

- **효과 및 의의**  
  - GPU가 처리해야 할 데이터가 많아도 안정적인 메모리 관리를 통해 **OOM 오류**를 방지함.  
  - QLoRA의 핵심 기술인 **4비트 양자화**와 함께 사용하면 메모리 효율성을 극대화함.  
  - **LoRA**와 **QLoRA**의 모든 핵심 개념을 이해하기 위해 필요한 주요 요소임.  

>**역전파의 메모리 사용량이 더 많은 이유**
>
>    - 각 레이어마다 **출력값 + 기울기(gradient)** 를 모두 저장합니다.  
>    - 기울기는 보통 **출력값과 동일한 크기**이므로,  
>    -  **순전파 대비 메모리 사용량이 2배 이상**입니다.

---

### 코드로 QLoRA 모델 활용하기

- **사용 도구:**  
  - **Hugging Face**의 코드와 **bitsandbytes** 라이브러리를 사용하면 QLoRA 학습을 간단히 수행 가능함.  

**예제 5.14. 4비트 양자화 모델 불러오기**

- **BitsAndBytesConfig 클래스 주요 옵션:**  
  - `load_in_4bit`: 모델을 **4비트 형식**으로 불러올지 여부  
  - `bnb_4bit_quant_type`: **4비트 데이터 타입** 지정  
  - `bnb_4bit_use_double_quant`: **2차 양자화**를 사용할지 여부  
  - `bnb_4bit_compute_dtype`: 계산 시 사용할 **데이터 형식** 지정  

> **Tips**
>
> **NF4(NormalFloat 4-bit)**: NF4는 정규 분포를 따르는 데이터의 효율적인 양자화를 위해 설계된 4비트 데이터 형식입니다. 전통적인 양자화 방식은 값의 분포를 고르게 나누지만, NF4는 정규 분포의 특성을 고려하여 각 양자화 구간에 동일한 수의 값을 할당하는 분위수 양자화를 사용합니다. 이를 통해 4비트 정밀도로도 모델의 중요한 정보를 효율적으로 보존할 수 있습니다. 
>
> **2차 양자화**는 모델의 가중치를 양자화할 때 사용되는 '양자화 상수(quantization constant)'를 추가로 양자화하는 기법을 의미합니다. 딥러닝 모델의 양자화는 모델의 가중치나 활성화 값을 낮은 비트 정밀도로 변환하여 메모리 사용량과 연산 복잡도를 줄이는 방법입니다. 이 과정에서, 가중치 값을 특정 범위로 정규화하기 위해 '양자화 상수'를 사용합니다. '2차 양자화'는 이러한 양자화 상수 자체를 다시 양자화하여 추가적인 메모리 절감을 도모하는 기술입니다. 예를 들어, QLoRA(Quantized Low-Rank Adaptation)와 같은 기법에서는 4비트 양자화를 사용하여 모델의 가중치를 압축합니다. 이때, 가중치 블록의 최대 절대값을 '양자화 상수'로 사용하며, 이러한 상수를 FP32(32비트 부동소수점) 형식으로 저장하면 추가적인 메모리 사용이 발생합니다. '2차 양자화'는 이 양자화 상수를 FP8(8비트 부동소수점) 형식으로 다시 양자화하여, 전체적인 메모리 사용량을 더욱 감소시키는 역할을 합니다.
  

In [16]:
# BitsAndBytesConfig 클래스 임포트
from transformers import BitsAndBytesConfig

# 4비트 양자화 설정 구성
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,              # 4비트 양자화 사용
    bnb_4bit_quant_type="nf4",      # 4비트 데이터 타입을 nf4로 지정
    bnb_4bit_use_double_quant=True, # 2차 양자화 사용
    bnb_4bit_compute_dtype=torch.bfloat16  # 계산시 bfloat16 형식 사용
)

# 양자화된 모델 로드
model_nf4 = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=nf4_config)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

**예제 5.15. 예제 5.11에서 QLoRA 모델을 불러오는 부분을 추가**

- **양자화 설정:**  
  - `BitsAndBytesConfig` 클래스를 사용해 **4비트 양자화** 및 **2차 양자화**를 정의함.  
  - 설정은 `bnb_config` 변수에 저장됨.  

- **모델 불러오기 및 학습 준비:**  
  - `prepare_model_for_kbit_training()` 함수를 호출하여 모델을 불러오고 학습 준비 과정을 수행함.  

- **LoRA 설정 적용 및 파라미터 확인:**  
  - `get_peft_model()` 함수를 사용해 **LoRA 설정**을 적용함.  
  - `print_trainable_parameters()` 메서드를 사용하여 학습 가능한 파라미터 수를 확인함.  

In [17]:
# 필요한 라이브러리 임포트
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

def load_model_and_tokenizer(model_id, peft=None):
    # 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    # PEFT 없이 기본 모델 로드
    if peft is None:
        model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map={"":0})

    # LoRA 적용
    elif peft == 'lora':
        model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map={"":0})
        # LoRA 설정
        lora_config = LoraConfig(
                    r=8,                            # LoRA의 랭크
                    lora_alpha=32,                  # LoRA의 스케일링 파라미터
                    target_modules=["query_key_value"], # LoRA를 적용할 모듈
                    lora_dropout=0.05,              # LoRA 드롭아웃 비율
                    bias="none",                    # 바이어스 학습 여부
                    task_type="CAUSAL_LM"          # 태스크 타입
                )

        model = get_peft_model(model, lora_config)
        model.print_trainable_parameters()
    
    # QLoRA 적용
    elif peft == 'qlora':
        # LoRA 설정
        lora_config = LoraConfig(
                    r=8,
                    lora_alpha=32,
                    target_modules=["query_key_value"],
                    lora_dropout=0.05,
                    bias="none",
                    task_type="CAUSAL_LM"
                )
        # 4비트 양자화 설정
        bnb_config = BitsAndBytesConfig(
                  load_in_4bit=True,               # 4비트 양자화 사용
                  bnb_4bit_use_double_quant=True,  # 이중 양자화 사용
                  bnb_4bit_quant_type="nf4",       # NF4 양자화 타입 사용
                  bnb_4bit_compute_dtype=torch.float16  # 연산 시 FP16 사용
              )
        # 양자화된 모델 로드 및 QLoRA 적용
        # 양자화된 모델 로드 
        model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
        # 그레이디언트 체크포인팅 활성화
        model.gradient_checkpointing_enable()
        # 4비트 학습을 위한 모델 준비
        model = prepare_model_for_kbit_training(model)
        # LoRA 적용
        model = get_peft_model(model, lora_config)
        # 학습 가능한 파라미터 출력
        model.print_trainable_parameters()

    # GPU 메모리 사용량 출력
    print_gpu_utilization()
    return model, tokenizer

**예제 5.16. QLoRA를 적용했을 때 GPU 메모리 사용량 확인**

- **모델 불러오기 시 메모리 사용량 비교:**  
  - 기존 방식: 모델을 불러올 때 **2.6GB** 메모리 필요.  
  - **QLoRA 사용 시:** 메모리 사용량이 절반 이하인 **1.183GB**로 감소.  

- **그레이디언트와 옵티마이저 상태 저장 시 메모리 감소:**  
  - 기존 방식보다 GPU 메모리 사용량이 줄어들어 총 사용량이 **1.722GB**로 크게 감소.  

In [18]:
cleanup()
print_gpu_utilization()

gpu_memory_experiment(batch_size=16, peft='qlora')

torch.cuda.empty_cache()

GPU 메모리 사용량: 0.828 GB
배치 사이즈: 16


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

trainable params: 1,572,864 || all params: 1,333,383,168 || trainable%: 0.11796039111242178
GPU 메모리 사용량: 1.878 GB
GPU 메모리 사용량: 2.417 GB
GPU 메모리 사용량: 0.828 GB


---

## 정리
- **GPU 메모리 효율화 학습 방법:**  
  - GPU 메모리에 올라가는 데이터 종류와 데이터 타입, 양자화에 대해 학습.  
  - 주요 사용 메모리: **모델 파라미터**, **그레이디언트**, **옵티마이저 상태**, **순전파 상태**  
  - 일반적으로 **16비트**(fp16, bf16) 데이터 사용 → 더 큰 모델에는 **8비트** 또는 **4비트**로 메모리 절약.  

- **단일 GPU에서의 학습 최적화:**  
  - **그레이디언트 누적(Gradient Accumulation):**  
    - 제한된 GPU 메모리에서도 큰 배치 크기로 학습 가능.  
  - **그레이디언트 체크포인팅(Checkpointing):**  
    - 순전파 상태를 저장하지 않아 **GPU 메모리 사용량 절감**.  

- **다중 GPU 분산 학습:**  
  - **모델 병렬화:** 모델을 여러 GPU에 나눠서 올림.  
  - **데이터 병렬화:** 동일한 모델을 여러 GPU에 올려 데이터만 나눠 학습.  
    - 문제점: 중복 데이터로 인해 메모리 비효율 발생.  
  - **ZeRO 방식:** 중복 데이터를 줄여 GPU 메모리를 효율적으로 사용.  

- **대형 모델의 미세 조정(Finetuning)과 PEFT 방식:**  
  - 모델 전체를 업데이트하기 어렵기 때문에 **PEFT 방식**이 활발히 연구됨.  
  - **LoRA:** 기존 모델에 학습 파라미터를 추가해 메모리 사용을 최소화.  
  - **QLoRA:** LoRA에 **4비트 양자화**를 추가해 더 높은 메모리 효율을 제공.  

- **다음 학습 내용(예고):**  
  - 4장과 5장의 내용을 종합해, 데이터를 기반으로 **SQL**을 생성하는 **Text2SQL** 작업을 수행할 예정.  

---

### **참고 자료**  

- 허깅페이스 1개의 GPU에서 모델 효율적으로 학습하기 공식 문서: [https://huggingface.co/docs/transformers/v4.29.1/en/perf_train_gpu_one](https://huggingface.co/docs/transformers/v4.29.1/en/perf_train_gpu_one)  
- 허깅페이스 여러 개의 GPU에서 모델 효율적으로 학습하기 공식 문서: [https://huggingface.co/docs/transformers/v4.29.1/en/perf_train_gpu_many](https://huggingface.co/docs/transformers/v4.29.1/en/perf_train_gpu_many)  
- 허깅페이스에서 제공하는 모델 메모리 사용량 계산기: [https://huggingface.co/spaces/hf-accelerate/model-memory-usage](https://huggingface.co/spaces/hf-accelerate/model-memory-usage)  
- GPU 통합 메모리: [https://developer.nvidia.com/blog/unified-memory-cuda-beginners/](https://developer.nvidia.com/blog/unified-memory-cuda-beginners/)  
- **LoRA 논문**: [https://arxiv.org/abs/2106.09685](https://arxiv.org/abs/2106.09685)  
- **QLoRA 논문**: [https://arxiv.org/abs/2305.14314](https://arxiv.org/abs/2305.14314)  
- **ZeRO 논문**: [https://arxiv.org/abs/1910.02054](https://arxiv.org/abs/1910.02054)  
- **기타**  
  - [https://medium.com/tensorflow/fitting-larger-networks-into-memory-583e3c758ff9](https://medium.com/tensorflow/fitting-larger-networks-into-memory-583e3c758ff9)  
  - [https://www.youtube.com/watch?v=LR3BmWCg7Y0&t=2861s&ab_channel=MLOpsLearners](https://www.youtube.com/watch?v=LR3BmWCg7Y0&t=2861s&ab_channel=MLOpsLearners)  

---